In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import cv2
import os
import shutil

## Загрузка данных датасета

In [2]:
# Загрузка датасетов
attr_df = pd.read_csv('list_attr_celeba.csv')
identity_df = pd.read_csv('identity_CelebA.csv', delimiter=' ', header=None, names=['image_id', 'id'])
bbox_df = pd.read_csv('list_bbox_celeba.csv')

landmarks_df = pd.read_csv('list_landmarks_celeba_original.csv', delimiter=r'\s+', skiprows=1, header=0)
landmarks_df = landmarks_df.reset_index()
landmarks_df = landmarks_df.rename(columns={'index': 'image_id'})

## Создаём тренировочный датасет

In [3]:
# Объединяем таблицы датасетов
merged_df = pd.merge(attr_df, identity_df[['image_id', 'id']], on='image_id', how='left').merge(bbox_df, on='image_id', how='left').merge(landmarks_df, on='image_id', how='left')

In [4]:
merged_df.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,165,184,244,176,196,249,194,271,266,260
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,140,204,220,204,168,254,146,289,226,289
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,244,104,264,105,263,121,235,134,251,140
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,796,539,984,539,930,687,762,756,915,756
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,273,169,328,161,298,172,283,208,323,207


In [9]:
# Инфо объединенного датасета
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202599 entries, 0 to 202598
Data columns (total 56 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   image_id             202599 non-null  object
 1   5_o_Clock_Shadow     202599 non-null  int64 
 2   Arched_Eyebrows      202599 non-null  int64 
 3   Attractive           202599 non-null  int64 
 4   Bags_Under_Eyes      202599 non-null  int64 
 5   Bald                 202599 non-null  int64 
 6   Bangs                202599 non-null  int64 
 7   Big_Lips             202599 non-null  int64 
 8   Big_Nose             202599 non-null  int64 
 9   Black_Hair           202599 non-null  int64 
 10  Blond_Hair           202599 non-null  int64 
 11  Blurry               202599 non-null  int64 
 12  Brown_Hair           202599 non-null  int64 
 13  Bushy_Eyebrows       202599 non-null  int64 
 14  Chubby               202599 non-null  int64 
 15  Double_Chin          202599 non-nu

In [6]:
# Отфильтруем картинки у которых слишком маленькое лицо (<40 пикселей) или слишком вытянутые (w/h или h/w > 3)
df_all = merged_df[~((merged_df['width'] < 40) | (merged_df['height'] < 40) | (merged_df['width'] / merged_df['height'] > 3 ) | (merged_df['height'] / merged_df['width'] > 3 ))]

In [10]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 202259 entries, 0 to 202598
Data columns (total 56 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   image_id             202259 non-null  object
 1   5_o_Clock_Shadow     202259 non-null  int64 
 2   Arched_Eyebrows      202259 non-null  int64 
 3   Attractive           202259 non-null  int64 
 4   Bags_Under_Eyes      202259 non-null  int64 
 5   Bald                 202259 non-null  int64 
 6   Bangs                202259 non-null  int64 
 7   Big_Lips             202259 non-null  int64 
 8   Big_Nose             202259 non-null  int64 
 9   Black_Hair           202259 non-null  int64 
 10  Blond_Hair           202259 non-null  int64 
 11  Blurry               202259 non-null  int64 
 12  Brown_Hair           202259 non-null  int64 
 13  Bushy_Eyebrows       202259 non-null  int64 
 14  Chubby               202259 non-null  int64 
 15  Double_Chin          202259 non-null  i

Отсеялось лишь 340 картинок из 200000+ датасета

In [12]:
# Выбираем датасет
TARGET_SAMPLES = 10000
MIN_SAMPLES_PER_ID = 25
MAX_SAMPLES_PER_ID = 25  # Ограничим максимум снимков на человека
N_UNIQUE_IDS = 400

id_counts = df_all['id'].value_counts()
selected_indices = []

Будем отсеивать и выравнивать количество мужчина/женщина и молодой/старый, чтобы модель была одинаково подготовлена к этим основным различиям в лицах.

In [13]:
male_count = 0
female_count = 0
young_count = 0
old_count = 0
for id_num in id_counts.index:
    id_samples = df_all[df_all['id'] == id_num]
    n_to_take = min(MIN_SAMPLES_PER_ID, len(id_samples))
    sample = id_samples.sample(n_to_take, random_state=42)
    # selected_indices.extend(sample.index.tolist())
    
    is_male_sample = (sample.iloc[0]['Male'] > 0).astype(np.bool)
    is_young_sample = (sample.iloc[0]['Young'] > 0).astype(np.bool)
    
    if is_male_sample and is_young_sample:
        if male_count < N_UNIQUE_IDS // 2 and young_count < N_UNIQUE_IDS // 2 and sample['Male'].sum() == n_to_take and sample['Young'].sum() == n_to_take:
            selected_indices.extend(sample.index.tolist())
            male_count += 1
            young_count += 1
        else:
            continue

    if not is_male_sample and is_young_sample:
        if female_count < N_UNIQUE_IDS // 2 and young_count < N_UNIQUE_IDS // 2 and sample['Male'].sum() == -n_to_take and sample['Young'].sum() == n_to_take:
            selected_indices.extend(sample.index.tolist())
            female_count += 1
            young_count += 1
        else:
            continue

    if is_male_sample and not is_young_sample:
        if male_count < N_UNIQUE_IDS // 2 and old_count < N_UNIQUE_IDS // 2 and sample['Male'].sum() == n_to_take and sample['Young'].sum() == -n_to_take:
            selected_indices.extend(sample.index.tolist())
            male_count += 1
            old_count += 1
        else:
            continue

    if not is_male_sample and not is_young_sample:
        if female_count < N_UNIQUE_IDS // 2 and old_count < N_UNIQUE_IDS // 2 and sample['Male'].sum() == -n_to_take and sample['Young'].sum() == -n_to_take:
            selected_indices.extend(sample.index.tolist())
            female_count += 1
            old_count += 1
        else:
            continue


    

In [14]:
len(selected_indices)

10000

In [15]:
df_train = df_all.loc[selected_indices]

In [17]:
# 3. Проверка баланса по атрибутам (и пост-корректировка при необходимости)
print("Баланс по атрибутам в финальной выборке:")
print(df_train[['Male', 'Young', 'Eyeglasses', 'Wearing_Hat', 'No_Beard']].mean()) 

Баланс по атрибутам в финальной выборке:
Male           0.0000
Young          0.0000
Eyeglasses    -0.7856
Wearing_Hat   -0.9060
No_Beard       0.6478
dtype: float64


Фотографии лиц с солнечными очками, шляпами и бородой встречаются реже, чем без них. Хорошо, что они всё таки есть в датасете, чтобы модель понимала как с ними работать, но и хорошо, что их небольшое количество, т.к. чаще работаем с открытыми лицами.

In [20]:
df_train.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,lefteye_x,lefteye_y,righteye_x,righteye_y,nose_x,nose_y,leftmouth_x,leftmouth_y,rightmouth_x,rightmouth_y
179285,179286.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,824,128,859,125,828,152,826,173,863,168
173217,173218.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,291,254,378,258,339,306,297,354,370,357
178573,178574.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,395,181,476,181,432,228,390,262,481,266
178018,178019.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,891,591,1173,597,1035,741,909,897,1179,903
174008,174009.jpg,-1,1,1,1,-1,-1,-1,-1,-1,...,253,269,384,271,318,338,253,396,382,396


In [21]:
df_train['id'].value_counts()

id
2013    25
7939    25
2362    25
4403    25
9398    25
        ..
4740    25
8968    25
3745    25
2820    25
3782    25
Name: count, Length: 400, dtype: int64

In [18]:
# Пути к папкам
source_folder = 'dataset'
target_folder = 'train_dataset'

# Создаем целевую папку, если она не существует
os.makedirs(target_folder, exist_ok=True)

# Копируем файлы
for image_id in df_train['image_id']:
    # Полный путь к исходному файлу
    source_path = os.path.join(source_folder, image_id)
    
    # Полный путь к целевому файлу
    target_path = os.path.join(target_folder, image_id)
    
    # Проверяем, существует ли исходный файл
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        # print(f"Скопировано: {image_id}")
    else:
        print(f"Файл не найден: {image_id}")

In [19]:
df_train.to_csv("./train_dataset/train_dataset.csv", index=True)

## Создаём валидационный датасет

Валидационный датасет создаётся из тех же id лиц, что и тренировочный, но из других картинок

In [20]:
VAL_DATASET_LEN = 1000

In [21]:
selected_indices_val = []

In [22]:
remaining_val = VAL_DATASET_LEN - len(selected_indices_val)
for id_num in id_counts.index:
    if remaining_val <= 0:
        break
    if id_num not in df_train['id'].values:
        continue
    id_samples = df_all[(df_all['id'] == id_num) & (~df_all.index.isin(selected_indices)) & (~df_all.index.isin(selected_indices_val))]
    if len(id_samples) > 0:
        n_possible = min(3, len(id_samples))
        # n_possible = 1
        n_to_take = min(n_possible, remaining_val)
        selected_indices_val.extend(id_samples.sample(n_to_take, random_state=42).index.tolist())
        remaining_val -= n_to_take

df_val = df_all.loc[selected_indices_val]

In [23]:
df_val['id'].value_counts()

id
8249    3
3782    3
2820    3
3745    3
8968    3
       ..
5472    3
5008    3
9177    3
5239    3
1161    1
Name: count, Length: 334, dtype: int64

In [24]:
# 3. Проверка баланса по атрибутам (и пост-корректировка при необходимости)
print("Баланс по атрибутам в финальной выборке:")
print(df_val[['Male', 'Young', 'Eyeglasses', 'Wearing_Hat', 'No_Beard']].mean())

Баланс по атрибутам в финальной выборке:
Male          -0.124
Young          0.200
Eyeglasses    -0.832
Wearing_Hat   -0.932
No_Beard       0.654
dtype: float64


In [25]:
# Пути к папкам
source_folder = 'dataset'
target_folder = 'val_dataset'

# Создаем целевую папку, если она не существует
os.makedirs(target_folder, exist_ok=True)

# Копируем файлы
for image_id in df_val['image_id']:
    # Полный путь к исходному файлу
    source_path = os.path.join(source_folder, image_id)
    
    # Полный путь к целевому файлу
    target_path = os.path.join(target_folder, image_id)
    
    # Проверяем, существует ли исходный файл
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        # print(f"Скопировано: {image_id}")
    else:
        print(f"Файл не найден: {image_id}")

In [26]:
df_val.to_csv("./val_dataset/val_dataset.csv", index=True)

## Создаём тестовый датасет для Identification Rate Metric

Тестовый датасет нужен для подсчёт метрики Identification Rate Metric и состоит из query и distractor частей, id которых не участвовали в тренировке.

In [27]:
train_ids = set(df_train['id'].unique())
all_ids = set(df_all['id'].unique())

In [28]:
single_counts = df_all['id'].value_counts()
single_ids = set(single_counts[(single_counts == 1) | (single_counts == 2)].index.tolist())

In [29]:
len(all_ids), len(train_ids)

(10177, 400)

In [30]:
unseen_ids = all_ids - train_ids
print(f"Всего непересекающихся id: {len(unseen_ids)}")

Всего непересекающихся id: 9777


In [31]:
# Количество уникальных id, которое будет выбрано для query и distractor
query_ids_count = 50  # Пример: ~10% для query
distractor_ids_count = 500 - 3 * query_ids_count

In [32]:
np.random.seed(1)  # Для воспроизводимости
shuffled_unseen_ids = np.random.permutation(list(unseen_ids - single_ids))

In [33]:
query_ids = shuffled_unseen_ids[:query_ids_count]
distractor_ids = shuffled_unseen_ids[query_ids_count:(query_ids_count + distractor_ids_count)]

print(f"Query set: {len(query_ids)} id")
print(f"Distractor set: {len(distractor_ids)} id")

Query set: 50 id
Distractor set: 350 id


In [34]:
query_images = []

for ids in query_ids:
    # Выбираем несколько изображений для этого id
    id_images = df_all[df_all['id'] == ids]['image_id'].values
    # Берем 1-3 случайных изображения
    num_query_per_id = min(3, len(id_images))
    selected_images = np.random.choice(id_images, num_query_per_id, replace=False)
    
    for img in selected_images:
        query_images.append({'image_id': img, 'id': ids})

df_query = pd.DataFrame(query_images)

In [35]:
df_query['id'].nunique(), df_query['id'].count()

(50, np.int64(150))

In [36]:
distractor_images = []

for ids in distractor_ids:
    # Выбираем одно изображение для этого id
    id_images = df_all[df_all['id'] == ids]['image_id'].values
    if len(id_images) > 0:
        selected_image = np.random.choice(id_images, 1)[0]
        distractor_images.append({'image_id': selected_image, 'id': ids})

df_distractors = pd.DataFrame(distractor_images)

In [37]:
df_distractors['id'].nunique(), df_distractors['id'].count()

(350, np.int64(350))

In [38]:
df_query['set_type'] = 'query'
df_distractors['set_type'] = 'distractor'
df_test = pd.concat([df_query, df_distractors], ignore_index=True)
df_test = pd.merge(df_all, df_test[['image_id', 'set_type']], on='image_id', how='right')

In [39]:
# Пути к папкам
source_folder = 'dataset'
target_folder = 'test_dataset'

# Создаем целевую папку, если она не существует
os.makedirs(target_folder, exist_ok=True)
os.makedirs(os.path.join(target_folder, 'query'), exist_ok=True)
os.makedirs(os.path.join(target_folder, 'distractor'), exist_ok=True)

# Копируем файлы
for image_id in df_test['image_id']:

    set_type = df_test[df_test['image_id'] == image_id]['set_type'].item()
    
    # Полный путь к исходному файлу
    source_path = os.path.join(source_folder, image_id)
    
    # Полный путь к целевому файлу
    target_path = os.path.join(target_folder, set_type, image_id)
    
    # Проверяем, существует ли исходный файл
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        # print(f"Скопировано: {image_id}")
    else:
        print(f"Файл не найден: {image_id}")

In [40]:
# Сохраняем результаты
df_query.to_csv('./test_dataset/query_set.csv', index=False)
df_distractors.to_csv('./test_dataset/distractor_set.csv', index=False)
df_test.to_csv('./test_dataset/test_dataset.csv', index=False)